In [27]:
import pandas as pd
import numpy as np

In [28]:
events_data = pd.read_csv('data/event_data_train.zip')
submission_data = pd.read_csv('data/submissions_data_train.zip')

events_data.head()

,step_id,timestamp,action,user_id
0,32815,1434340848,viewed,17632
1,32815,1434340848,passed,17632
2,32815,1434340848,discovered,17632
3,32811,1434340895,discovered,17632
4,32811,1434340895,viewed,17632


In [29]:
events_data['date'] = pd.to_datetime(events_data.timestamp, unit = 's')
events_data['day'] = events_data.date.dt.date

submission_data['date'] = pd.to_datetime(submission_data.timestamp, unit = 's')
submission_data['day'] = submission_data.date.dt.date

users_data = events_data.groupby('user_id', as_index= False)\
        .agg({'timestamp': 'max'})\
        .rename({'timestamp':'last_timestamp'}, axis = 'columns')

now = 1526772811
drop_out_threasold = 2592000

users_data['is_gone_user'] = (now - users_data.last_timestamp) > drop_out_threasold
users_data.head()


,user_id,last_timestamp,is_gone_user
0,1,1472827464,True
1,2,1519226966,True
2,3,1444581588,True
3,5,1499859939,True
4,7,1521634660,True


In [30]:
submission_data.head(3)

,step_id,timestamp,submission_status,user_id,date,day
0,31971,1434349275,correct,15853,2015-06-15 06:21:15,2015-06-15
1,31972,1434348300,correct,15853,2015-06-15 06:05:00,2015-06-15
2,31972,1478852149,wrong,15853,2016-11-11 08:15:49,2016-11-11


In [31]:
submission_data.submission_status.unique()

array(['correct', 'wrong'], dtype=object)

In [32]:
users_scores = submission_data.pivot_table(index='user_id',\
                        columns = 'submission_status',\
                        values = 'step_id',\
                        aggfunc = 'count',\
                        fill_value = 0)\
                        .reset_index()
users_data = users_data.merge(users_scores, on = 'user_id', how = 'outer')
users_data = users_data.fillna(0)
users_data.head()

,user_id,last_timestamp,is_gone_user,correct,wrong
0,1,1472827464,True,0.0,0.0
1,2,1519226966,True,2.0,0.0
2,3,1444581588,True,29.0,23.0
3,5,1499859939,True,2.0,2.0
4,7,1521634660,True,0.0,0.0


In [33]:
users_invent_data = events_data.pivot_table(index = 'user_id',
                        columns = 'action',
                        values  = 'step_id',
                        aggfunc = 'count',
                        fill_value = 0).reset_index()
users_data = users_data.merge(users_invent_data, how = 'outer')

# как собрать число дней - тупо посчитать число уникальных дней
users_days = events_data.groupby('user_id').day.nunique()

users_days.to_frame().reset_index()
users_data = users_data.merge(users_days, on = 'user_id', how = 'outer')
users_data['passed_corse'] = users_data.passed > 170

In [34]:
users_data.head()

,user_id,last_timestamp,is_gone_user,correct,wrong,discovered,passed,started_attempt,viewed,day,passed_corse
0,1,1472827464,True,0.0,0.0,1,0,0,1,1,False
1,2,1519226966,True,2.0,0.0,9,9,2,10,2,False
2,3,1444581588,True,29.0,23.0,91,87,30,192,7,False
3,5,1499859939,True,2.0,2.0,11,11,4,12,2,False
4,7,1521634660,True,0.0,0.0,1,1,0,1,1,False


In [35]:
# идем дальше
# 1. добавляем столбец когда в 1-й раз появился
first_timestamp = events_data.groupby(['user_id'], as_index=False)\
                .agg({'timestamp':'min'})\
                .rename(columns={'timestamp':'first_timestamp'})
first_timestamp.head()


,user_id,first_timestamp
0,1,1472827464
1,2,1514383364
2,3,1434358476
3,5,1466156809
4,7,1521634660


In [36]:
users_data = users_data.merge(first_timestamp, on = 'user_id',  how = 'outer')
users_data.head()

,user_id,last_timestamp,is_gone_user,correct,wrong,discovered,passed,started_attempt,viewed,day,passed_corse,first_timestamp
0,1,1472827464,True,0.0,0.0,1,0,0,1,1,False,1472827464
1,2,1519226966,True,2.0,0.0,9,9,2,10,2,False,1514383364
2,3,1444581588,True,29.0,23.0,91,87,30,192,7,False,1434358476
3,5,1499859939,True,2.0,2.0,11,11,4,12,2,False,1466156809
4,7,1521634660,True,0.0,0.0,1,1,0,1,1,False,1521634660


In [37]:
# трюк - пересортируем колонки
# users_data.reindex(['user_id', 'first_timestamp'] + list(users_data.columns[1:-1].values), axis=1)

In [38]:
users_data.shape

(19234, 12)

In [39]:
### отбираем из events_data только события, произошедшие в 1е 3 дня
# events_data.head()

In [40]:
min_max_timestamp = events_data.groupby(['user_id'], as_index=False)\
                    .agg({'timestamp':'min'})\
                    .rename(columns={'timestamp':'min_timestamp'})

min_max_timestamp['max_timestamp'] = min_max_timestamp['min_timestamp'] + 3*24*60*60


# теперь нам больше не нужен таймстемп начала учебы - 
min_max_timestamp.drop(['min_timestamp'], axis=1, inplace=True)
min_max_timestamp.head()

,user_id,max_timestamp
0,1,1473086664
1,2,1514642564
2,3,1434617676
3,5,1466416009
4,7,1521893860


In [41]:
# теперь берем все большую таблицу events_data
df_tmp = pd.merge(events_data, min_max_timestamp, on='user_id', how='left',
         left_index=False, right_index=False, sort=False,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

df_train = df_tmp[df_tmp['timestamp'] <= df_tmp['max_timestamp']]

In [42]:
df_train[df_train['user_id']==17632]

,step_id,timestamp,action,user_id,date,day,max_timestamp
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48,2015-06-15,1434600048
1,32815,1434340848,passed,17632,2015-06-15 04:00:48,2015-06-15,1434600048
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48,2015-06-15,1434600048
3,32811,1434340895,discovered,17632,2015-06-15 04:01:35,2015-06-15,1434600048
4,32811,1434340895,viewed,17632,2015-06-15 04:01:35,2015-06-15,1434600048
...,...,...,...,...,...,...,...
73137,31959,1434565384,viewed,17632,2015-06-17 18:23:04,2015-06-17,1434600048
73214,31976,1434565645,viewed,17632,2015-06-17 18:27:25,2015-06-17,1434600048
73233,31976,1434565678,passed,17632,2015-06-17 18:27:58,2015-06-17,1434600048
73245,31971,1434565711,viewed,17632,2015-06-17 18:28:31,2015-06-17,1434600048


In [ ]:
### плохой путь урезания исходного датафрейма

#df = pd.DataFrame()

#treshold = 3*24*60*60

#for user_id in events_data.user_id.unique():
#    min_time = events_data[events_data['user_id'] == user_id]['timestamp'].min()
#    min_timestamp = min_time + treshold
#    df_tmp = events_data.query("(user_id == @user_id) & (timestamp < @min_timestamp) ")
#    df.append(df_tmp)



In [ ]:
### плохой путь урезания исходного датафрейма

#df = pd.DataFrame()

#treshold = 3*24*60*60

#for user_id in events_data.user_id.unique():
#    min_time = users_data[users_data['user_id'] == user_id]['min_timestamp']
#    min_timestamp = min_time + treshold
#    df_tmp = events_data.query("(user_id == @user_id) & (timestamp < @min_timestamp) ")
#    df.append(df_tmp)


In [43]:
# максимальное число дней, которые пользователь был на курсе. Мы отсекли всех, кто за окном в 3 суток
df_train.groupby('user_id', as_index=False).day.nunique().max()
df_train.shape

(1014985, 7)

In [44]:
# мы теперь хоти как-то сингеннерить фичи так, чтобы предсказывать уход или не уход
# у нас сейчас есть данные по поведению пользователей за первые 3 дня. и есть таблица значений ушел или нет
df_train_clean = df_train.drop('max_timestamp', axis=1)
df_train_clean.head()


,step_id,timestamp,action,user_id,date,day
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48,2015-06-15
1,32815,1434340848,passed,17632,2015-06-15 04:00:48,2015-06-15
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48,2015-06-15
3,32811,1434340895,discovered,17632,2015-06-15 04:01:35,2015-06-15
4,32811,1434340895,viewed,17632,2015-06-15 04:01:35,2015-06-15


In [45]:
df_train_clean['date'] = pd.to_datetime(df_train_clean['timestamp'], unit ='s')


In [47]:
df_train_clean['day'] = df_train_clean['date'].dt.date
df_train_clean.head()

,step_id,timestamp,action,user_id,date,day
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48,2015-06-15
1,32815,1434340848,passed,17632,2015-06-15 04:00:48,2015-06-15
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48,2015-06-15
3,32811,1434340895,discovered,17632,2015-06-15 04:01:35,2015-06-15
4,32811,1434340895,viewed,17632,2015-06-15 04:01:35,2015-06-15


In [52]:
usc_train = df_train_clean.pivot_table(index = 'user_id',
                        columns = 'action',
                        values  = 'step_id',
                        aggfunc = 'count',
                        fill_value = 0).reset_index()
usc_train.shape, users_data.shape

((19234, 5), (19234, 12))

In [55]:
df = pd.merge(usc_train, users_data[['user_id','passed_corse']], on='user_id', how='outer',
         left_index=False, right_index=False, sort=False,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)
df.head()

,user_id,discovered,passed,started_attempt,viewed,passed_corse
0,1,1,0,0,1,False
1,2,9,9,2,9,False
2,3,15,15,4,20,False
3,5,1,1,0,1,False
4,7,1,1,0,1,False


In [56]:
df.to_csv('data/my_contest.csv')